In [35]:
from openai import OpenAI
client = OpenAI()
from pydub import AudioSegment

import simpleaudio.functionchecks as fc
from pydub.playback import play
from collections import defaultdict

In [36]:
audio_file_advertisement = open("advertisement_steve.wav", "rb")

# script1 = open("script1_steve.m4a", "rb")
advertisement_transcript = client.audio.transcriptions.create(
  file=audio_file_advertisement,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["word"]
)

print(advertisement_transcript)



Transcription(text='And now a word from our sponsors. Peter, do you ever find yourself tired after a long day of class? Wanna pick me up to get you through your day? Look no further than Koopa Cafe outside Green Library. This student favorite is a Stanford staple and provides everything you need from breakfast wraps to a warm tiger spice chai. Stop by today for a sweet treat. Thanks for listening, Peter. And now back to the podcast.', task='transcribe', language='english', duration=26.149999618530273, words=[{'word': 'And', 'start': 0.0, 'end': 0.36000001430511475}, {'word': 'now', 'start': 0.36000001430511475, 'end': 0.5600000023841858}, {'word': 'a', 'start': 0.5600000023841858, 'end': 0.8799999952316284}, {'word': 'word', 'start': 0.8799999952316284, 'end': 0.8799999952316284}, {'word': 'from', 'start': 0.8799999952316284, 'end': 1.0800000429153442}, {'word': 'our', 'start': 1.0800000429153442, 'end': 1.3799999952316284}, {'word': 'sponsors', 'start': 1.3799999952316284, 'end': 1.77

In [37]:
SAMPLE_NAME = "Peter"

advertisement_times_to_replace = []

for word in advertisement_transcript.words: 
    if str.lower(word["word"]) == str.lower(SAMPLE_NAME): 
        advertisement_times_to_replace.append([word["start"] * 1000, word["end"] * 1000])


print(advertisement_times_to_replace)
advertisement_times_to_replace[0][0] -= 250
advertisement_times_to_replace[0][1] += 125

advertisement_times_to_replace[1][0] -= 250

advertisement_audio_raw = AudioSegment.from_wav("advertisement_steve.wav")
print(advertisement_times_to_replace)


# advertisement_times_to_replace = [[2410.0000858306885, 2785.0000858306885], [23969.999313354492, 24219.999313354492]]

[[2660.0000858306885, 2660.0000858306885], [24219.999313354492, 24219.999313354492]]
[[2410.0000858306885, 2785.0000858306885], [23969.999313354492, 24219.999313354492]]


In [38]:
# play(advertisement_audio_raw)

advertisement_audio_slices = []

audio_left = advertisement_audio_raw

prev_end = 0 # where the last one left off 

for times in advertisement_times_to_replace:
    lower_bound = times[0]
    upper_bound = times[1]

    before_name = advertisement_audio_raw[prev_end:lower_bound]
    advertisement_audio_slices.append(before_name)
    advertisement_audio_slices.append("NAME")
    prev_end = upper_bound

advertisement_audio_slices.append(advertisement_audio_raw[prev_end + 100:])
print(advertisement_audio_slices)


[<pydub.audio_segment.AudioSegment object at 0x7fd1c83ab460>, 'NAME', <pydub.audio_segment.AudioSegment object at 0x7fd1c83ab190>, 'NAME', <pydub.audio_segment.AudioSegment object at 0x7fd1c83ab340>]


Now we need to find all of the sub names for the advertisement audio

In [39]:
audio_file_advertisement_names = open("advertisement_names.wav", "rb")
advertisement_names_transcript = client.audio.transcriptions.create(
  file=audio_file_advertisement_names,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["word"]
)

print(advertisement_names_transcript)
advertisement_audio_names_raw = AudioSegment.from_wav("advertisement_names.wav")

Transcription(text="I'm changing this to a word. And now a word from our sponsors. Carla, do you ever find yourself tired after a long day of class? Stop by today for a sweet treat. Thanks for listening, Carla. And now back to the podcast. And now a word from our sponsors. Joseph, do you ever find yourself tired after a long day of class? Stop by today for a sweet treat. Thanks for listening Joseph. And now back to the podcast. And now a word from our sponsors. Christopher, do you ever find yourself tired after a long day of class? Stop by today for a sweet treat. Thanks for listening, Christopher. Stop by today for a sweet treat. Thanks for live. God dammit. Stop by today for a sweet treat. Thanks for listening, Christopher. And now back to the podcast. And now a word from our sponsors. Manish, do you ever find yourself tired after a long day of class? Stop by today for a sweet treat. Thanks for listening, Manish. And now back to the podcast. And now a word from our sponsors. RB, do y

In [10]:
advertisement_name_times = {"carla": [], "joseph": [], "christopher": [], "r.b.": [], "mahira": []}

sub_names = set(advertisement_name_times.keys())
# sub_names.add("carla")



for word in advertisement_names_transcript.words: 
    if str.lower(word["word"]) in sub_names:
        lower_name = str.lower(word["word"]) 
        # print(advertisement_name_subs[lower_name])
        
        advertisement_name_times[lower_name].append([word["start"] * 1000, word["end"] * 1000])
        # advertisement_times_to_replace.append([word["start"] * 1000, word["end"] * 1000])
        
# print(advertisement_name_times)



In [40]:
advertisement_name_times = {"carla": [], "joseph": [], "christopher": [], "manish": [], "mahira": []}

sub_names = set(advertisement_name_times.keys())

# sub_names.add("carla")
temp = []

for word in advertisement_names_transcript.words:
    
    if str.lower(word["word"]) in sub_names:
        lower_name = str.lower(word["word"])
        
        
        # print(advertisement_name_subs[lower_name])
        
        advertisement_name_times[lower_name].append([word["start"] * 1000, word["end"] * 1000])
        # advertisement_times_to_replace.append([word["start"] * 1000, word["end"] * 1000])



advertisement_name_audio_slices = {"carla": [], "joseph": [], "christopher": [], "manish": [], "mahira": []}
advertisement_name_times = {'carla': [[13899.999618530273, 13899.999618530273], [21299.999237060547, 21299.999237060547]], 'joseph': [[27180.000686645508, 28099.999237060547], [35040.0016784668, 35419.99816894531]], 'christopher': [[40679.99954223633, 41419.998931884766], [57639.998626708984, 58309.99984741211]], 'r.b.': [], 'mahira': [[120959.99908447266, 121220.00122070312], [127879.99725341797, 128199.99694824219]]}


# advertisement_name_times["christopher"].pop(1)

advertisement_name_times["christopher"][0][0] -= 300
advertisement_name_times["christopher"][0][1] += 300

advertisement_name_times["christopher"][1][0] -= 300
advertisement_name_times["christopher"][1][1] += 150

advertisement_name_times["joseph"][0][0] -= 100
advertisement_name_times["joseph"][0][1] += 300
advertisement_name_times["joseph"][1][0] -= 200
# advertisement_name_times["joseph"][1][1] += 300


print(advertisement_name_times)
# advertisement_name_times["r.b."][0][0] -= 400
# advertisement_name_times["r.b."][0][1] += 300

# advertisement_name_times["r.b."][1][0] -= 75
# advertisement_name_times["r.b."][1][1] += 150

advertisement_name_times["mahira"][0][0] -= 400
advertisement_name_times["mahira"][0][1] += 300

advertisement_name_times["mahira"][1][0] -= 75
advertisement_name_times["mahira"][1][1] += 150

# advertisement_name_times["manish"][0][0] -= 400
# advertisement_name_times["manish"][0][1] += 300

# advertisement_name_times["manish"][1][0] -= 75
# advertisement_name_times["manish"][1][1] += 150


# SOURCE OF TRUTH 
advertisement_name_times = {'carla': [[13899.999618530273, 13899.999618530273], [21299.999237060547, 21299.999237060547]], 'joseph': [[27180.000686645508, 28199.999237060547], [34840.0016784668, 35419.99816894531]], 'christopher': [[40379.99954223633, 41519.998931884766], [57639.998626708984, 58259.99984741211]], 'r.b.': [], 'mahira': [[120959.99908447266, 121220.00122070312], [127879.99725341797, 128199.99694824219]], 'manish': [[63559.999084472656, 64340.000915527344], [71179.99725341797, 71789.99938964844]]}

# good: joseph

for name in advertisement_name_times.keys(): 
    times = advertisement_name_times[name]

    for i in range(len(times)):
        time = times[i] 
        lower_bound = time[0]
        upper_bound = time[1]
        
        cur_slice = advertisement_audio_names_raw[lower_bound:upper_bound]
        advertisement_name_audio_slices[name].append(cur_slice)

play(advertisement_name_audio_slices["manish"][1])

# IN CASE CELLS ABOVE RE RUN
# advertisement_times_to_replace = [[40979.99954223633, 41119.998931884766], [47479.99954223633, 47740.0016784668], [57939.998626708984, 58159.99984741211]]



{'carla': [[13899.999618530273, 13899.999618530273], [21299.999237060547, 21299.999237060547]], 'joseph': [[27080.000686645508, 28399.999237060547], [34840.0016784668, 35419.99816894531]], 'christopher': [[40379.99954223633, 41719.998931884766], [57339.998626708984, 58459.99984741211]], 'r.b.': [], 'mahira': [[120959.99908447266, 121220.00122070312], [127879.99725341797, 128199.99694824219]]}


Add them together given input

In [41]:
user_input = input("Type the name you'd like to insert. Your options are: Christopher, Joseph, Mahira, Maneesh")
name = str.lower(user_input)
if name == "maneesh": 
    name = "manish"

# assume chosen the advertisement story
audio_arr = advertisement_audio_slices
name_arr = advertisement_name_audio_slices[name]

final_audio = audio_arr[0]
next_name_idx = 0
# print(audio_arr)
for i in range(1, len(audio_arr)): 
    if audio_arr[i] == "NAME": 
        
        final_audio += name_arr[next_name_idx]
        next_name_idx += 1

    else: 
        final_audio += audio_arr[i]

play(final_audio)





Now let's move onto the audio story.

In [43]:
audio_file_story = open("story_steve.wav", "rb")

# script1 = open("script1_steve.m4a", "rb")
story_transcript = client.audio.transcriptions.create(
  file=audio_file_story,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["word"]
)


In [44]:
SAMPLE_NAME = "Steve"

story_times_to_replace = []

for word in story_transcript.words: 
    if str.lower(word["word"]) == str.lower(SAMPLE_NAME): 
        story_times_to_replace.append([word["start"] * 1000, word["end"] * 1000])


print(story_times_to_replace)
story_times_to_replace[0][0] -= 100
story_times_to_replace[0][1] += 25 

story_times_to_replace[1][0] -= 60
story_times_to_replace[1][1] += 25 
print(story_times_to_replace)



story_audio_raw = AudioSegment.from_wav("story_steve.wav")
# play(song)

[[8600.000381469727, 8819.999694824219], [16959.999084472656, 17219.999313354492]]
[[8500.000381469727, 8844.999694824219], [16899.999084472656, 17244.999313354492]]


In [45]:
# play(advertisement_audio_raw)

story_audio_slices = []

audio_left = story_audio_raw

prev_end = 0 # where the last one left off 
# print(story_times_to_replace)
story_times_to_replace = [[8390.000381469727, 8894.999694824219], [16599.999084472656, 17344.999313354492]]



print(story_times_to_replace)


for times in story_times_to_replace:
    lower_bound = times[0]
    upper_bound = times[1]

    before_name = story_audio_raw[prev_end:lower_bound]
    story_audio_slices.append(before_name)
    story_audio_slices.append("NAME")
    prev_end = upper_bound

story_audio_slices.append(story_audio_raw[prev_end:])
print(story_audio_slices)
# play(story_audio_slices[4])

[[8390.000381469727, 8894.999694824219], [16599.999084472656, 17344.999313354492]]
[<pydub.audio_segment.AudioSegment object at 0x7fd1a8095c40>, 'NAME', <pydub.audio_segment.AudioSegment object at 0x7fd1a8095c70>, 'NAME', <pydub.audio_segment.AudioSegment object at 0x7fd1e913a6a0>]


In [46]:
audio_file_story_names = open("story_names.wav", "rb")
story_names_transcript = client.audio.transcriptions.create(
  file=audio_file_story_names,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["word"]
)
story_audio_names_raw = AudioSegment.from_wav("story_names.wav")

In [32]:
story_name_times = {"carla": [], "joseph": [], "manish": [], "mahira": []}

sub_names = set(story_name_times.keys())


for word in story_names_transcript.words:
    
    if str.lower(word["word"]) in sub_names:
        lower_name = str.lower(word["word"])        
        story_name_times[lower_name].append([word["start"] * 1000, word["end"] * 1000])


story_name_audio_slices = {"carla": [], "joseph": [], "manish": [], "mahira": []}




# SOURCE OF TRUTH 
story_name_times = {'carla': [[5209.999847412109, 6010.000133514404], [10659.99984741211, 11620.000076293945]], 'joseph': [[17800.9993133545, 18620.000457763672], [23500.000228881836, 24560.000228881836]], 'manish': [[53419.99816894531, 54540.0016784668], [58580.00030517578, 59760.001373291016]], 'mahira': [[117279.99877929688, 118259.99755859375], [121875.99694824219, 122700.00091552734]]}


for name in story_name_times.keys(): 
    times = story_name_times[name]

    for i in range(len(times)):
        time = times[i] 
        lower_bound = time[0]
        upper_bound = time[1]
        
        cur_slice = story_audio_names_raw[lower_bound:upper_bound]
        story_name_audio_slices[name].append(cur_slice)

# IN CASE CELLS ABOVE RE RUN

play(story_name_audio_slices["mahira"][1])


{'carla': [[5209.999847412109, 6010.000133514404], [10659.99984741211, 11620.000076293945]], 'joseph': [[17800.9993133545, 18620.000457763672], [23500.000228881836, 24560.000228881836]], 'manish': [[53419.99816894531, 54540.0016784668], [58580.00030517578, 59760.001373291016]], 'mahira': [[117279.99877929688, 118259.99755859375], [121875.99694824219, 122700.00091552734]]}


In [34]:
user_input = input("Type the name you'd like to insert. Your options are: Carla, Joseph, Mahira, Maneesh")
name = str.lower(user_input)
if name == "maneesh": 
    name = "manish"

# assume chosen the advertisement story
audio_arr = story_audio_slices
name_arr = story_name_audio_slices[name]

final_audio = audio_arr[0]
next_name_idx = 0
# print(audio_arr)
for i in range(1, len(audio_arr)): 
    if audio_arr[i] == "NAME": 
        
        final_audio += name_arr[next_name_idx]
        next_name_idx += 1

    else: 
        final_audio += audio_arr[i]

play(final_audio)
